In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
pip install -i https://test.pypi.org/simple/ my_krml_25076922==2024.0.3.3

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

import os

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
path ='/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/data/processed'

sales_data = pd.read_csv(f'{path}/df_pred.csv')

In [5]:
sales_data.head()

,item_id,store_id,date,total_sales,day_of_week,month,year
0,HOBBIES_1_001,CA_1,2011-01-29,0.0,5,1,2011
1,HOBBIES_1_002,CA_1,2011-01-29,0.0,5,1,2011
2,HOBBIES_1_003,CA_1,2011-01-29,0.0,5,1,2011
3,HOBBIES_1_004,CA_1,2011-01-29,0.0,5,1,2011
4,HOBBIES_1_005,CA_1,2011-01-29,0.0,5,1,2011


In [6]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47107050 entries, 0 to 47107049
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   item_id      object 
 1   store_id     object 
 2   date         object 
 3   total_sales  float64
 4   day_of_week  int64  
 5   month        int64  
 6   year         int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 2.5+ GB


In [7]:
from sklearn.preprocessing import LabelEncoder

le_item = LabelEncoder()
le_store = LabelEncoder()

sales_data['item_id'] = le_item.fit_transform(sales_data['item_id'])
sales_data['store_id'] = le_store.fit_transform(sales_data['store_id'])

In [8]:
sales_data.head()

,item_id,store_id,date,total_sales,day_of_week,month,year
0,1437,0,2011-01-29,0.0,5,1,2011
1,1438,0,2011-01-29,0.0,5,1,2011
2,1439,0,2011-01-29,0.0,5,1,2011
3,1440,0,2011-01-29,0.0,5,1,2011
4,1441,0,2011-01-29,0.0,5,1,2011


In [20]:
from sklearn.preprocessing import StandardScaler

X = sales_data[['item_id', 'store_id', 'day_of_week', 'month', 'year']]
y = sales_data['total_sales']

scaler = StandardScaler()
X_scaled = scaler.fit(X)

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
import xgboost as xgb

model = xgb.XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [13]:
# Make predictions on the validation set
y_pred = model.predict(X_val)

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Evaluate the model
rmse = mean_squared_error(y_val, y_pred, squared=False)  # RMSE
mae = mean_absolute_error(y_val, y_pred)

print(f"RMSE Validation: {rmse}")
print(f"MAE Validation: {mae}")

RMSE Validation: 8.867779334911672
MAE Validation: 4.103078436951001


/Users/vega7unk/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [15]:
model_1 = xgb.XGBRegressor(
    n_estimators=100,          # Number of trees
    learning_rate=0.1,         # Step size shrinkage
    max_depth=6,               # Maximum tree depth for base learners
    subsample=0.8,             # Subsample ratio of the training instance
    colsample_bytree=0.8,      # Subsample ratio of columns when constructing each tree
    random_state=42
)

# Fit the model on the training data
model_1.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=True)


/Users/vega7unk/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:9.17839
[1]	validation_0-rmse:9.15781
[2]	validation_0-rmse:9.13933
[3]	validation_0-rmse:9.12405
[4]	validation_0-rmse:9.11308
[5]	validation_0-rmse:9.09797
[6]	validation_0-rmse:9.09206
[7]	validation_0-rmse:9.07932
[8]	validation_0-rmse:9.07041
[9]	validation_0-rmse:9.06282
[10]	validation_0-rmse:9.05481
[11]	validation_0-rmse:9.04816
[12]	validation_0-rmse:9.04101
[13]	validation_0-rmse:9.03525
[14]	validation_0-rmse:9.03237
[15]	validation_0-rmse:9.02621
[16]	validation_0-rmse:9.02028
[17]	validation_0-rmse:9.01500
[18]	validation_0-rmse:9.00900
[19]	validation_0-rmse:9.00729
[20]	validation_0-rmse:9.00428
[21]	validation_0-rmse:9.00041
[22]	validation_0-rmse:8.99747
[23]	validation_0-rmse:8.99530
[24]	validation_0-rmse:8.99313
[25]	validation_0-rmse:8.99146
[26]	validation_0-rmse:8.98916
[27]	validation_0-rmse:8.98649
[28]	validation_0-rmse:8.98512
[29]	validation_0-rmse:8.98248
[30]	validation_0-rmse:8.98081
[31]	validation_0-rmse:8.97909
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [16]:
y_pred_1 = model_1.predict(X_val)

In [17]:
# Evaluate the model
rmse = mean_squared_error(y_val, y_pred_1, squared=False)  # RMSE
mae = mean_absolute_error(y_val, y_pred_1)

print(f"RMSE Validation: {rmse}")
print(f"MAE Validation: {mae}")

RMSE Validation: 8.91885900411708
MAE Validation: 4.129478078824783


/Users/vega7unk/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [18]:
import joblib

joblib.dump(model, '/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/models/predictive/xgb_predictive_model.pkl')

['/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/models/predictive/xgb_predictive_model.pkl']

In [21]:
joblib.dump(scaler, '/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/models/predictive/scaler.pkl')

['/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/models/predictive/scaler.pkl']

In [23]:
import pickle

with open("/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/models/predictive/le_store.pkl", "wb") as f:
    pickle.dump(le_store, f)

with open("/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/models/predictive/le_item.pkl", "wb") as f:
    pickle.dump(le_item, f)